# Projeto 1 - Ciência dos Dados

Nome: Arthur Gomes Chieppe

Nome: João Pedro Di Jardim Marquez

**Atenção:** Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
Carregando algumas bibliotecas:

In [1]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
from math import *
from emoji import UNICODE_EMOJI
pd.set_option("display.max_rows", None, "display.max_columns", None)

**Em `filename`, coloque o nome do seu arquivo de dados!**

In [2]:
import os

filename = 'PS5.xlsx'
if filename in os.listdir():
    print(f'Encontrei o arquivo {filename}, tudo certo para prosseguir com a prova!')
else:
    print(f'Não encontrei o arquivo {filename} aqui no diretório {os.getcwd()}, será que você não baixou o arquivo?')

Encontrei o arquivo PS5.xlsx, tudo certo para prosseguir com a prova!


Carregando a base de dados com os tweets classificados como relevantes e não relevantes:

In [3]:
train = pd.read_excel(filename, sheet_name = 'Treinamento')
train.head(5)

,Tweets,Relevância:
0,@immandy4 o arquiteto do ps5 disse que ele ser...,1
1,o vídeo que eu fiz pro evento tá no youtube tb...,0
2,@chief117_com_br @vingadorbrambz enquanto o da...,0
3,possivelmente hoje sai o valor e a data de lan...,1
4,"@lealemy_ sim, mas os produtos da sony vão par...",1


In [4]:
test = pd.read_excel(filename, sheet_name = 'Teste')
test.head(5)


,Tweets,Relevância:
0,"sony nega, mas ações continuam em queda após r...",1
1,hoje tem \n\n#ps5,1
2,@prodigyrr não faça isso. ps5 ta uma completa ...,0
3,"hoje lançam o preço do ps5, tô só por saber qu...",1
4,"@1playergirl bom, nesse momento me sinto num h...",0


___
## Classificador automático de sentimento


Faça aqui uma descrição do seu produto e o que considerou como relevante ou não relevante na classificação dos tweets.

Nosso produto trata-se do console de videogames PlayStation5. Quando extraímos os Tweets, havia uma espera coletiva acerca da livestream de lançamento do novo PlayStation 5. Assim , consideramos relevante especulações acerca do preço do novo console, bem como opiniões dos usuários sobre seu design. Ademais, consideramos algumas comparações entre o novo PlayStation e Xbox como relevantes, porém somente aquelas bem fundamentadas e intencionadas, já que consideramos irrelevantes comparações vagas e com discurso de ódio. Anúncios feitos acerca da live também foram consideradas como relevantes, com exceção de anúncios que tinham como objetivo somente a autopromoção dos usuários.

___
### Montando um Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

In [5]:
#Limpando caracteres:
def limpa_tweets(dataframe):
    train_limpo = dataframe.iloc[:, 0]
    ls= []
    for w in train_limpo:
        w = w.casefold()
        w = w.replace(":",'')
        w = w.replace('"','')
        w = w.replace("'",'')
        w = w.replace("(",'')
        w = w.replace(")",'')
        w = w.replace('!','')
        w = w.replace('...','')
        w = w.replace(',','')
        w = w.replace('\n','')
        w = w.replace('/','')
        w = w.replace(' a ',' ')
        w = w.replace(' e ',' ')
        w = w.replace(' o ',' ')
        w = w.replace(' de ',' ')
        w = w.replace(' do ',' ')
        w = w.replace(' no ',' ')
        w = w.replace(' na ',' ')
        w = w.replace(' os ',' ')
        w = w.replace(' que ',' ')
        w = w.replace(' q ',' ')
        w = w.replace(' as ',' ')
        w = w.replace(' - ',' ')
        w = " ".join(filter(lambda x:x[0]!='@', w.split()))
        w = ''.join(' ' + char+' ' if char in UNICODE_EMOJI else char for char in w)
        w = w.replace('  ',' ')
        w = w.strip()
        ls.append(w)
    dataframe['Tweets'] = ls
    return dataframe
train = limpa_tweets(train)


In [6]:
#train.to_excel(r'PS5Emoji.xlsx', sheet_name='Treinamento', index = False)

## Classificador:





In [7]:
#Calculando Probabilidades:
total_portugues = 50000 #Consideramos todas as palavras do portugues comumente usadas no twitter para maximizar smoothing.
def treinamento(dataframe):
    Rele = pd.Series(dtype=object)
    Irrele = pd.Series(dtype=object)
    i = 0
    for s in dataframe['Tweets']:
        lsWords = pd.Series(s.split())
        if dataframe['Relevância:'][i] == 1:
            Rele = Rele.append(lsWords)
        else:
            Irrele = Irrele.append(lsWords)
        i += 1
    
    #Calcaulando probabilidade em tweets relevantes:
    lsRele = []
    for v in Rele.value_counts():
        lsRele.append(log10((v+1)/(total_portugues+Rele.count())))
    lsKeysRele = Rele.value_counts().index.tolist()
    dic_rele = {lsKeysRele[i]: lsRele[i] for i in range(len(lsKeysRele))} 
    
    lsIrrele = []
    for v in Irrele.value_counts():
        lsIrrele.append(log10((v+1)/(total_portugues+Irrele.count())))
    lsKeysIrrele = Irrele.value_counts().index.tolist()
    dic_irrele = {lsKeysIrrele[i]: lsIrrele[i] for i in range(len(lsKeysIrrele))}
    print(dic_irrele)
    return [dic_rele, dic_irrele, Rele, Irrele]



In [8]:
treino = treinamento(train)
dic_rele = treinamento(train)[0]
dic_irrele = treinamento(train)[1]
Rele = treinamento(train)[2]
Irrele = treinamento(train)[3]

def classifica(dataframe, dic_rele, dic_irrele, Rele, Irrele):
    lsClassificacao = []
    for t in dataframe['Tweets']:
        lsWord = t.split()
        lsRele = []
        lsIrrele = []
        for w in lsWord:
            if w in dic_rele:
                lsRele.append(dic_rele[w])
            if w in dic_irrele:
                lsIrrele.append(dic_irrele[w])
            else:
                lsRele.append(log10(1/(total_portugues)))
                lsIrrele.append(log10(1/(total_portugues)))
        if (log10(Rele.count()/(total_portugues+Rele.count())) + np.sum(lsRele)) <= (log10(Irrele.count()/(total_portugues+Irrele.count())) + np.sum(lsIrrele)):
            lsClassificacao.append(1)
        else:
            lsClassificacao.append(0)
    return lsClassificacao
teste_treinamento = pd.Series(classifica(train, dic_rele, dic_irrele, Rele, Irrele))
train['Classificação'] = teste_treinamento


{'ps5': -2.6875537320392184, 'um': -2.9428262371425244, 'é': -2.9885837277031997, 'dia': -3.0977281971282675, 'pra': -3.141193890909358, 'bom': -3.189498570483913, 'eu': -3.2438562328065057, 'não': -3.2585794896272122, 'vai': -3.2585794896272122, 'hoje': -3.273819456183949, '#ps5': -3.30600413955535, 'evento': -3.340766245814562, 'me': -3.3592496515085752, 'uma': -3.3592496515085752, 'com': -3.3592496515085752, 'da': -3.3592496515085752, 'se': -3.3987581927922488, 'tem': -3.442223886573339, 'xbox': -3.490528566147894, 'para': -3.490528566147894, 'mais': -3.57484945184793, 'preço': -3.57484945184793, 'meu': -3.57484945184793, 'ter': -3.6070341352193314, 'isso': -3.6070341352193314, 'vou': -3.641796241478543, 'ou': -3.641796241478543, 'já': -3.679584802367943, 'ser': -3.679584802367943, 'por': -3.679584802367943, 'sony': -3.679584802367943, 'mas': -3.679584802367943, 'em': -3.679584802367943, 'tá': -3.679584802367943, 'ver': -3.7209774875261683, 'ps5?': -3.7209774875261683, 'dar': -3.720

{'ps5': -2.6875537320392184, 'um': -2.9428262371425244, 'é': -2.9885837277031997, 'dia': -3.0977281971282675, 'pra': -3.141193890909358, 'bom': -3.189498570483913, 'eu': -3.2438562328065057, 'não': -3.2585794896272122, 'vai': -3.2585794896272122, 'hoje': -3.273819456183949, '#ps5': -3.30600413955535, 'evento': -3.340766245814562, 'me': -3.3592496515085752, 'uma': -3.3592496515085752, 'com': -3.3592496515085752, 'da': -3.3592496515085752, 'se': -3.3987581927922488, 'tem': -3.442223886573339, 'xbox': -3.490528566147894, 'para': -3.490528566147894, 'mais': -3.57484945184793, 'preço': -3.57484945184793, 'meu': -3.57484945184793, 'ter': -3.6070341352193314, 'isso': -3.6070341352193314, 'vou': -3.641796241478543, 'ou': -3.641796241478543, 'já': -3.679584802367943, 'ser': -3.679584802367943, 'por': -3.679584802367943, 'sony': -3.679584802367943, 'mas': -3.679584802367943, 'em': -3.679584802367943, 'tá': -3.679584802367943, 'ver': -3.7209774875261683, 'ps5?': -3.7209774875261683, 'dar': -3.720

{'ps5': -2.6875537320392184, 'um': -2.9428262371425244, 'é': -2.9885837277031997, 'dia': -3.0977281971282675, 'pra': -3.141193890909358, 'bom': -3.189498570483913, 'eu': -3.2438562328065057, 'não': -3.2585794896272122, 'vai': -3.2585794896272122, 'hoje': -3.273819456183949, '#ps5': -3.30600413955535, 'evento': -3.340766245814562, 'me': -3.3592496515085752, 'uma': -3.3592496515085752, 'com': -3.3592496515085752, 'da': -3.3592496515085752, 'se': -3.3987581927922488, 'tem': -3.442223886573339, 'xbox': -3.490528566147894, 'para': -3.490528566147894, 'mais': -3.57484945184793, 'preço': -3.57484945184793, 'meu': -3.57484945184793, 'ter': -3.6070341352193314, 'isso': -3.6070341352193314, 'vou': -3.641796241478543, 'ou': -3.641796241478543, 'já': -3.679584802367943, 'ser': -3.679584802367943, 'por': -3.679584802367943, 'sony': -3.679584802367943, 'mas': -3.679584802367943, 'em': -3.679584802367943, 'tá': -3.679584802367943, 'ver': -3.7209774875261683, 'ps5?': -3.7209774875261683, 'dar': -3.720

{'ps5': -2.6875537320392184, 'um': -2.9428262371425244, 'é': -2.9885837277031997, 'dia': -3.0977281971282675, 'pra': -3.141193890909358, 'bom': -3.189498570483913, 'eu': -3.2438562328065057, 'não': -3.2585794896272122, 'vai': -3.2585794896272122, 'hoje': -3.273819456183949, '#ps5': -3.30600413955535, 'evento': -3.340766245814562, 'me': -3.3592496515085752, 'uma': -3.3592496515085752, 'com': -3.3592496515085752, 'da': -3.3592496515085752, 'se': -3.3987581927922488, 'tem': -3.442223886573339, 'xbox': -3.490528566147894, 'para': -3.490528566147894, 'mais': -3.57484945184793, 'preço': -3.57484945184793, 'meu': -3.57484945184793, 'ter': -3.6070341352193314, 'isso': -3.6070341352193314, 'vou': -3.641796241478543, 'ou': -3.641796241478543, 'já': -3.679584802367943, 'ser': -3.679584802367943, 'por': -3.679584802367943, 'sony': -3.679584802367943, 'mas': -3.679584802367943, 'em': -3.679584802367943, 'tá': -3.679584802367943, 'ver': -3.7209774875261683, 'ps5?': -3.7209774875261683, 'dar': -3.720

{'ps5': -2.6875537320392184, 'um': -2.9428262371425244, 'é': -2.9885837277031997, 'dia': -3.0977281971282675, 'pra': -3.141193890909358, 'bom': -3.189498570483913, 'eu': -3.2438562328065057, 'não': -3.2585794896272122, 'vai': -3.2585794896272122, 'hoje': -3.273819456183949, '#ps5': -3.30600413955535, 'evento': -3.340766245814562, 'me': -3.3592496515085752, 'uma': -3.3592496515085752, 'com': -3.3592496515085752, 'da': -3.3592496515085752, 'se': -3.3987581927922488, 'tem': -3.442223886573339, 'xbox': -3.490528566147894, 'para': -3.490528566147894, 'mais': -3.57484945184793, 'preço': -3.57484945184793, 'meu': -3.57484945184793, 'ter': -3.6070341352193314, 'isso': -3.6070341352193314, 'vou': -3.641796241478543, 'ou': -3.641796241478543, 'já': -3.679584802367943, 'ser': -3.679584802367943, 'por': -3.679584802367943, 'sony': -3.679584802367943, 'mas': -3.679584802367943, 'em': -3.679584802367943, 'tá': -3.679584802367943, 'ver': -3.7209774875261683, 'ps5?': -3.7209774875261683, 'dar': -3.720

In [9]:
teste_treinamento.value_counts()

1    151
0    149
dtype: int64

In [10]:
#Comparando com as entradas manualmente digitadas:
train['Relevância:'].value_counts()

0    152
1    148
Name: Relevância:, dtype: int64

In [11]:
pd.crosstab(train['Relevância:'], train['Classificação'], normalize='index')

Classificação,0,1
Relevância:,,
0,0.973684,0.026316
1,0.006757,0.993243


___
### Verificando a performance do Classificador

Agora você deve testar o seu classificador com a base de Testes.

In [12]:
test = limpa_tweets(test)
teste_teste = pd.Series(classifica(test, dic_rele, dic_irrele, Rele, Irrele))

test['Classificação'] = teste_teste


In [13]:
teste_teste.value_counts()

1    112
0     88
dtype: int64

In [14]:
test['Relevância:'].value_counts()

1    110
0     90
Name: Relevância:, dtype: int64

In [15]:
pd.crosstab(test['Relevância:'], test['Classificação'], normalize=True)

Classificação,0,1
Relevância:,,
0,0.25,0.20
1,0.19,0.36


In [16]:
pd.crosstab(test['Relevância:'], test['Classificação'], normalize='index')

Classificação,0,1
Relevância:,,
0,0.555556,0.444444
1,0.345455,0.654545


In [17]:
pd.crosstab(test['Relevância:'], test['Classificação'], normalize='columns')

Classificação,0,1
Relevância:,,
0,0.568182,0.357143
1,0.431818,0.642857


In [18]:
dic_rele

{'ps5': -2.6578961839009247,
 'é': -3.1095703164081074,
 'sony': -3.154152449060848,
 'vai': -3.1660516723605556,
 'hoje': -3.178286128777567,
 'com': -3.178286128777567,
 'mais': -3.190875256085588,
 'da': -3.217204194807937,
 'um': -3.217204194807937,
 'preço': -3.275196141785624,
 'não': -3.275196141785624,
 'evento': -3.307380825157025,
 'dia': -3.342142931416237,
 'em': -3.342142931416237,
 'eu': -3.3606263371102503,
 'pra': -3.443600572175014,
 '#ps5': -3.467081668024537,
 'tem': -3.467081668024537,
 'para': -3.467081668024537,
 'se': -3.4919052517495692,
 'sobre': -3.4919052517495692,
 'uma': -3.4919052517495692,
 'será': -3.5182341904719183,
 'brasil': -3.5182341904719183,
 'por': -3.5182341904719183,
 'ps5.': -3.576226137449605,
 'jogos': -3.6084108208210064,
 'lançamento': -3.6084108208210064,
 'pro': -3.6084108208210064,
 'ser': -3.6084108208210064,
 'vão': -3.6431729270802182,
 '👍': -3.6431729270802182,
 'amanhã': -3.6431729270802182,
 'só': -3.6431729270802182,
 'of': -3.6

In [19]:
dic_irrele

{'ps5': -2.6875537320392184,
 'um': -2.9428262371425244,
 'é': -2.9885837277031997,
 'dia': -3.0977281971282675,
 'pra': -3.141193890909358,
 'bom': -3.189498570483913,
 'eu': -3.2438562328065057,
 'não': -3.2585794896272122,
 'vai': -3.2585794896272122,
 'hoje': -3.273819456183949,
 '#ps5': -3.30600413955535,
 'evento': -3.340766245814562,
 'me': -3.3592496515085752,
 'uma': -3.3592496515085752,
 'com': -3.3592496515085752,
 'da': -3.3592496515085752,
 'se': -3.3987581927922488,
 'tem': -3.442223886573339,
 'xbox': -3.490528566147894,
 'para': -3.490528566147894,
 'mais': -3.57484945184793,
 'preço': -3.57484945184793,
 'meu': -3.57484945184793,
 'ter': -3.6070341352193314,
 'isso': -3.6070341352193314,
 'vou': -3.641796241478543,
 'ou': -3.641796241478543,
 'já': -3.679584802367943,
 'ser': -3.679584802367943,
 'por': -3.679584802367943,
 'sony': -3.679584802367943,
 'mas': -3.679584802367943,
 'em': -3.679584802367943,
 'tá': -3.679584802367943,
 'ver': -3.7209774875261683,
 'ps5?':

___
### Concluindo

___
## Aperfeiçoamento:

Os trabalhos vão evoluir em conceito dependendo da quantidade de itens avançados:

* Limpar: \n, :, ", ', (, ), etc SEM remover emojis
* Corrigir separação de espaços entre palavras e emojis ou entre emojis e emojis
* Propor outras limpezas e transformações que não afetem a qualidade da informação ou classificação
* Criar categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante (3 categorias: C, mais categorias conta para B)
* Explicar por que não posso usar o próprio classificador para gerar mais amostras de treinamento
* Propor diferentes cenários para Naïve Bayes fora do contexto do projeto
* Sugerir e explicar melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* Montar um dashboard que realiza análise de sentimento e visualiza estes dados

___
## Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**